In [131]:
field = 'BootesB'
fname = '/Users/ytcheng/Downloads/BootesB.csv'

f = open(fname)
wdfname = f.readlines()[1].strip().split(',')
f.close()

df = pd.read_csv(fname, skiprows = 2,\
                   names = wdfname, low_memory = False) 

df = df[df['type']==6]

In [132]:
import astropy.io.fits as fits
import astropy.wcs as wcs
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky

def add_xy(field, df):
    order = [c for c in df.columns]
    # find the x, y solution with all quad
    for inst in [1,2]:
        hdrdir = '/Users/ytcheng/ciber/doc/20170617_Stacking/maps/astroutputs/inst' + str(inst) + '/'
        xoff = [0,0,512,512]
        yoff = [0,512,0,512]
        for iquad,quad in enumerate(['A','B','C','D']):
            hdulist = fits.open(hdrdir + field + '_' + quad + '_astr.fits')
            wcs_hdr=wcs.WCS(hdulist[('primary',1)].header, hdulist)
            hdulist.close()
            src_coord = SkyCoord(ra=df['ra']*u.degree, dec=df['dec']*u.degree, frame='icrs')

            x_arr, y_arr = wcs_hdr.all_world2pix(df['ra'],df['dec'],0)
            df['x' + quad] = x_arr + xoff[iquad]
            df['y' + quad] = y_arr + yoff[iquad]

        df['meanx'] = (df['xA'] + df['xB'] + df['xC'] + df['xD']) / 4
        df['meany'] = (df['yA'] + df['yB'] + df['yC'] + df['yD']) / 4

        # assign the x, y with the nearest quad solution
        df['x'+str(inst)] = df['xA'].copy()
        df['y'+str(inst)] = df['yA'].copy()
        bound = 511.5
        df.loc[ (df['meanx'] < bound) & (df['meany'] > bound),'x'+str(inst)] = df['xB']
        df.loc[ (df['meanx'] < bound) & (df['meany'] > bound),'y'+str(inst)] = df['yB']
        
        df.loc[ (df['meanx'] > bound) & (df['meany'] < bound),'x'+str(inst)] = df['xC']
        df.loc[ (df['meanx'] > bound) & (df['meany'] < bound),'y'+str(inst)] = df['yC']

        df.loc[ (df['meanx'] > bound) & (df['meany'] > bound),'x'+str(inst)] = df['xD']
        df.loc[ (df['meanx'] > bound) & (df['meany'] > bound),'y'+str(inst)] = df['yD']

    # write x, y to df
    order = order[:2] + ['x1','y1','x2','y2'] + order[2:]
    dfout = df[order].copy()
    
    return dfout

df = add_xy(field,df)

In [136]:
df['profPSF_tot'] = df[['probPSF_u','probPSF_g','probPSF_r','probPSF_i','probPSF_z']].sum(axis=1)

In [143]:
zdf = df[(df['x1']>-0.5) & (df['y1']>-0.5) & (df['x1']<1023.5) & (df['y1']<1023.5)]
df5 = df[df['profPSF_tot']==5]

In [73]:
df[df['type']==6]['']

,ra,dec,flags,type,Photz,PhotzErr,Specz,SpeczErr,zWarning,SpecClass,...,lnLStar_u,lnLStar_g,lnLStar_r,lnLStar_i,lnLStar_z,probPSF_u,probPSF_g,probPSF_r,probPSF_i,probPSF_z
8,218.198376,32.814005,68987912192,6,NaN,NaN,0.593292,0.000182,0.0,QSO,...,-0.265901,-2.134168,-31.057730,-31.645970,-8.828955,1,1,1,1,0
10,217.296564,33.161480,68987912192,6,NaN,NaN,1.116047,0.000595,0.0,QSO,...,-0.093348,-0.662732,-0.073043,-0.213037,-1.051310,1,1,1,1,1
12,217.505676,32.325242,96972308552,6,NaN,NaN,1.264269,0.000147,0.0,QSO,...,-0.122544,-1.605087,-0.457383,-8.104205,-0.601474,1,1,1,1,1
14,216.935173,33.641277,17661174088192,6,NaN,NaN,1.241259,0.000605,0.0,QSO,...,-0.700115,-22.405400,-0.967038,-0.391547,-0.031511,1,1,1,1,1
18,217.242698,33.810920,68987912192,6,NaN,NaN,0.000081,0.000117,4.0,STAR,...,-0.428331,-0.653282,-0.001764,-0.310243,-0.061683,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234872,217.614620,34.154721,17661174088192,6,NaN,NaN,0.191028,0.000014,0.0,GALAXY,...,-1.689076,-4.089736,-0.907094,-1.854560,-0.138374,1,1,1,1,0
234873,218.177818,34.188914,35253361180688,6,NaN,NaN,-0.000221,0.000020,0.0,STAR,...,-1.339046,-241.753200,-16.673730,-37.110230,-1.606757,1,1,1,1,1
234877,216.185744,34.093187,68988043520,6,NaN,NaN,3.312058,0.000462,0.0,QSO,...,-0.283836,-0.090877,-0.001375,-0.038067,-0.241271,0,1,1,1,1
234884,217.979797,32.363589,35253360132112,6,NaN,NaN,2.174437,0.000522,0.0,QSO,...,-1.169143,-0.633677,-0.233730,-0.072545,-0.360401,1,1,1,1,1


In [79]:
a=df[df['type']==6][['probPSF_u','probPSF_g','probPSF_r','probPSF_i','probPSF_z']].sum(axis=1)

In [82]:
len(a)/len(a[a==5])

4.220738045738046

In [14]:
df[df['type']==3][['type', 'SpecClass', 'SpecsubClass',
                  'u', 'g', 'r', 'i', 'z', 'probPSF',
                  'lnLStar_u', 'lnLStar_g', 'lnLStar_r', 
                  'lnLStar_i', 'lnLStar_z', 'probPSF_u', 
                  'probPSF_g', 'probPSF_r', 'probPSF_i',
                  'probPSF_z']].head(50)

,type,SpecClass,SpecsubClass,u,g,r,i,z,probPSF,lnLStar_u,lnLStar_g,lnLStar_r,lnLStar_i,lnLStar_z,probPSF_u,probPSF_g,probPSF_r,probPSF_i,probPSF_z
0,3,GALAXY,STARBURST,19.40215,18.32653,17.75009,17.32625,17.14962,0,-96.394810,-2967.218000,-4183.925000,-5487.19900,-483.210100,0,0,0,0,0
1,3,GALAXY,NaN,20.85009,18.60028,17.34149,16.86059,16.49603,0,-2.770620,-1238.448000,-4490.801000,-5142.77300,-802.597300,0,0,0,0,0
2,3,GALAXY,STARBURST,18.15889,16.99423,16.73831,16.60148,16.66307,0,-238.617300,-10215.660000,-5652.463000,-1525.84300,-162.120300,0,0,0,0,0
3,3,GALAXY,STARFORMING,19.67384,18.32643,17.69170,17.34905,17.06897,0,-63.694260,-3620.485000,-5078.080000,-4633.01500,-392.747700,0,0,0,0,0
4,3,GALAXY,NaN,19.81375,17.95576,17.33778,17.02204,16.73563,0,-2.097741,-2872.028000,-3296.314000,-2768.64900,-185.738500,0,0,0,0,0
5,3,GALAXY,NaN,18.63364,17.39460,16.99841,16.80915,16.68257,0,-201.411200,-6531.724000,-5736.079000,-3399.70500,-443.685100,0,0,0,0,0
6,3,GALAXY,NaN,20.73121,18.84303,17.77085,17.31844,16.93962,0,-2.987821,-1001.287000,-2620.761000,-2129.54200,-350.553000,0,0,0,0,0
7,3,GALAXY,NaN,21.57418,19.21135,17.74724,17.21152,16.87734,0,-0.721413,-551.325100,-2656.578000,-2795.09100,-473.248200,0,0,0,0,0
9,3,GALAXY,NaN,19.62935,17.83979,17.01086,16.62112,16.31843,0,-43.469970,-5273.552000,-7842.705000,-8277.05500,-1287.251000,0,0,0,0,0
11,3,GALAXY,NaN,22.26010,20.92881,19.12963,18.54012,18.19936,0,-0.000016,-6.936075,-329.030100,-465.95150,-19.566760,0,0,0,0,0
